# Homework 3 - Databases and SQL
In this guide, we will be connecting to the sqlite database created from the lecture, fill it with values, then run a few queries. You will be using the data files from the previous homework.

### Instructions
1. Follow the instructions on how to setup your Python and Jupyter (or VSCode) environment and cloning or downloading our repository. Instructions can be found in the class notes.
2. Fill the missing pieces of code in the provided notebook.
3. Answer the questions in the notebook through code.
4. Run the notebook and make sure everything works.


### Dataset Overview
We will use two datasets for this assignment. The first one is the same as used in HW1, which consists of four text files, each containing a story. Files are in the `Datasets` directory of this repository. The stories are:

- `story-1.txt`: The Monkey and the Crocodile
- `story-2.txt`: The Musical Donkey
- `story-3.txt`: A Tale of Three Fish
- `story-4.txt`: The Foolish Lion and the Clever Rabbit

The second dataset covers information about soccer players in sqlite format. This file is located in the `Datasets` directory of this repository. The file is called `fifa_soccer_dataset.sqlite.gz`.

**IMPORTANT** The database is compressed and needs to be decompressed before use. You can do this by running the following command in your terminal on Linux or MacOS:

```bash
gunzip Datasets/fifa_soccer_dataset.sqlite.gz
```

If you are using Windows, you can use the following command in your powershell:

```powershell
$sourceFile = "$PWD\Datasets\fifa_soccer_dataset.sqlite.gz"
$destinationFile = "$PWD\Datasets\fifa_soccer_dataset.sqlite"

$inputStream = [System.IO.File]::OpenRead($sourceFile)
$outputStream = [System.IO.File]::Create($destinationFile)
$gzipStream = New-Object System.IO.Compression.GzipStream($inputStream, [System.IO.Compression.CompressionMode]::Decompress)
$gzipStream.CopyTo($outputStream)

$gzipStream.Close()
$outputStream.Close()
$inputStream.Close()
```

Alternatively, you can extract the file using the GUI of your operating system.


### Submission Guidelines

- Submit your completed notebook as a HTML export, or a PDF file.

To export to HTML, if you are on Jupyter, select `File` > `Export Notebook As` > `HTML`.

If you are on VSCode, you can use the `Jupyter: Export to HTML` command.
 - Open the command palette (Ctrl+Shift+P or Cmd+Shift+P on Mac).
     - Search for `Jupyter: Export to HTML`.
     - Save the HTML file to your computer and submit it via Canvas.

---

### Decompress fifa dataset

```bash
$ gzip -d -c fifa_soccer_dataset.sqlite.gz > ../Database/fifa_soccer_dataset.sqlite
```

### Part 1 Story Analysis
First, we need to import the correct library to use sqlite functions. Can you figure out which library are we going to use?

In [1]:
import sqlite3 #Input needed Library here
import os

# Local directory
print(os.getcwd())

c:\Ricardo\2025-02 SP25 USABLE ARTIFICIAL INTELLIGENCE\GitHub\usable_ai\Homework


Now we have our functions ready to go, let's get the current path and connect to our database. For `dbPath` variable, create a db with name `mydb.sqlite`.

In [2]:
data_path = '../Datasets' # Path to the datasets folder
dbpath = '../Database/sqlite_hw4.db' # Select a path to save the database file
conn = sqlite3.connect(dbpath) 

Now connected, we can create our cursor variable using the `cursor()` function.

In [3]:
# Using https://docs.python.org/3/library/sqlite3.html#sqlite3-tutorial

cur = conn.cursor()

Using `cur`, we want to add in our new tables. However, if we add them multiple times, we will receive an error. So, we first need to `DROP` the tables if they exist already.<br>Enter your `DROP` queries below as an argument to the `execute` methods. Do this twice, once for the `stories` table and one for the `word_counts` table.

In [ ]:
drop_stories_query  = "DROP TABLE stories" # Input DROP query for stories table here
drop_info_query = "DROP TABLE information" # Input DROP query for information table here

#Now we run both queries
cur.execute(drop_stories_query)
cur.execute(drop_info_query)

# Removing word_counts
drop_word_counts_query = "DROP TABLE word_counts" # Input DROP query for word_counts table here
cur.execute(drop_word_counts_query)

# List all tables in the database
cur.execute("SELECT name FROM sqlite_master")
tables = cur.fetchall()

display(tables)


We now have a clean slate set up to create our new tables! Now, design the queries to create new tables for `stories` and `word_counts`. <br>
The `stories` table only needs a primary key `story_id` that is an integer and a text field called `story` to store the corrosoponding story text.
<br>
The `word_counts` table will need a little more. Please include the following:
- `word_id`: primary key, integer
- `word`: text
- `count`: integer
- `story_id`: integer, foreign key to `stories` table

In [4]:
# Defining story_id, info_id, word_id as primary keys

create_stories_query = "CREATE TABLE \
                            stories (story_id INTEGER PRIMARY KEY, \
                                    story TEXT)" # Insert CREATE TABLE query for stories table here

create_info_query =  "CREATE TABLE \
                            information (info_id INTEGER PRIMARY KEY, \
                                        details TEXT)" # Insert CREATE TABLE query for information table here

cur.execute(create_stories_query)
cur.execute(create_info_query)
conn.commit()

create_word_counts_query = "CREATE TABLE \
                word_counts (word_id INTEGER PRIMARY KEY, \
                            word TEXT, \
                            count INTEGER, \
                            story_id INTEGER NOT NULL, \
                            FOREIGN KEY (story_id) REFERENCES stories(story_id) ON DELETE CASCADE)" # CREATE TABLE word_counts

cur.execute(create_word_counts_query)
conn.commit()

In [5]:
# Check structure of tables created 

# List all tables in the database
cur.execute("SELECT name FROM sqlite_master")
tables = cur.fetchall()

display(tables)

[('stories',), ('information',), ('word_counts',)]

In [6]:
# Show the structure of each table

for table in tables:
    print(f"\nStructure of table: {table[0]}")
    cur.execute(f"PRAGMA table_info({table[0]});")
    columns = cur.fetchall()
    for column in columns:
        print(column)


Structure of table: stories
(0, 'story_id', 'INTEGER', 0, None, 1)
(1, 'story', 'TEXT', 0, None, 0)

Structure of table: information
(0, 'info_id', 'INTEGER', 0, None, 1)
(1, 'details', 'TEXT', 0, None, 0)

Structure of table: word_counts
(0, 'word_id', 'INTEGER', 0, None, 1)
(1, 'word', 'TEXT', 0, None, 0)
(2, 'count', 'INTEGER', 0, None, 0)
(3, 'story_id', 'INTEGER', 1, None, 0)


With our new empty tables ready, we can now loop through the stories and store the word counts.<br> Similar to how we looped through the stories in HW 1, we now have an additional step of inserting this data.<br>
In the cell below, please add the two queries for inserting these data rows. <br><br>
The first insert is for storing the story text in the `stories` table while reading from the file. The second insert is after counting up all the words, and is for inserting those values into the `word_counts` table. Remember to pass these query string variables into the `execute()` methods.

In [7]:
stories = ["story-1.txt", "story-2.txt","story-3.txt","story-4.txt"]
data_path = '..\Datasets' # Path to the datasets folder

print(data_path)
for story in stories:
    print(story)
    story_path = os.path.join(data_path, story)
    print(story_path)

..\Datasets
story-1.txt
..\Datasets\story-1.txt
story-2.txt
..\Datasets\story-2.txt
story-3.txt
..\Datasets\story-3.txt
story-4.txt
..\Datasets\story-4.txt


In [24]:
stories = ["story-1.txt", "story-2.txt","story-3.txt","story-4.txt"]  # Do not need / because os.path.join will do the magic

for story in stories:
    words = []
    count_of_each_word = {}
    story_id = ""
    i=0
    try:
        # Open the file
        story_path = os.path.join(data_path, story)
        with open(story_path,"r", encoding='utf-8') as fp:
            # reading data from file and splitting into words
            # and storing them in a list
            story_text = fp.read()
            
            # For the below query, you will need to use a '?' to
            #     represent where you want the story_text to be inputed.
            #     The actual text in story_text is passed in a tuple as 
            #     second parameter of the execute command().
            
            insert_story_query = "INSERT INTO stories (story) VALUES (?)" # Input INSERT query for the stories table here
            cur.execute(insert_story_query, (story_text,))
            
            #Grabbing the last id inserted, so we can use it when inserting values into the word_counts table
            story_id = cur.lastrowid
            conn.commit()
            words = story_text.split()
            
            # Close the file
            fp.close()
            
    except Exception as e:
        print("Unable to open the file: " + str(e))

    # Just like before, we are iterating over each word and using a dictonary to store the word counts
    for word in words:
        if(word in count_of_each_word):
            count_of_each_word[word] += 1
        else:
            count_of_each_word[word] = 1

    for key in count_of_each_word:
        insert_count_query = "INSERT INTO  word_counts (word, count, story_id) VALUES(?,?,?)" # Input query here, using '?' again in the VALUES () portion
        
        cur.execute(insert_count_query, (key, count_of_each_word[key], story_id))
        conn.commit()


In [ ]:
# Check table content

# Retrieve all records from the stories table
cur.execute("SELECT * FROM stories")
all_data_stories = cur.fetchall()

display(all_data_stories)

[(0,
  'The Monkey and the Crocodile:\n\nOnce upon a time, in a forest, there lived a monkey who resided on a jamoon (berry) tree, which was on the banks of a river. In the same forest, there lived a crocodile and his wife. One day, the crocodile came to the banks of the river and rested under the tree. The kindhearted monkey offered him some fruits. The crocodile came back the next day for more fruits, as he loved them. As days passed by, the crocodile and the monkey became good friends.\n\nOne day, the monkey sent some fruits for the crocodile’s wife. She ate the fruits and liked them, but was jealous, as she didn’t like her husband spending time with the monkey. She told her husband, “If the fruits are so juicy, I wonder how sweet the monkey’s heart would be. Get me the heart of the monkey.” The crocodile was not willing to kill his friend, but had no choice.\n\nHe invited the monkey to his house for dinner and that his wife would like to meet him. The monkey was happy, but couldn’t

In [27]:
# Retrieve all records from the word_counts table
cur.execute("SELECT * FROM word_counts")
all_data_word_counts = cur.fetchall()

display(all_data_word_counts)

[(1, 'The', 8, 1),
 (2, 'Monkey', 1, 1),
 (3, 'and', 8, 1),
 (4, 'the', 26, 1),
 (5, 'Crocodile:', 1, 1),
 (6, 'Once', 1, 1),
 (7, 'upon', 1, 1),
 (8, 'a', 6, 1),
 (9, 'time,', 1, 1),
 (10, 'in', 1, 1),
 (11, 'forest,', 2, 1),
 (12, 'there', 2, 1),
 (13, 'lived', 2, 1),
 (14, 'monkey', 9, 1),
 (15, 'who', 1, 1),
 (16, 'resided', 1, 1),
 (17, 'on', 4, 1),
 (18, 'jamoon', 1, 1),
 (19, '(berry)', 1, 1),
 (20, 'tree,', 1, 1),
 (21, 'which', 1, 1),
 (22, 'was', 5, 1),
 (23, 'banks', 2, 1),
 (24, 'of', 3, 1),
 (25, 'river.', 1, 1),
 (26, 'In', 1, 1),
 (27, 'same', 1, 1),
 (28, 'crocodile', 8, 1),
 (29, 'his', 6, 1),
 (30, 'wife.', 2, 1),
 (31, 'One', 2, 1),
 (32, 'day,', 2, 1),
 (33, 'came', 2, 1),
 (34, 'to', 5, 1),
 (35, 'river', 1, 1),
 (36, 'rested', 1, 1),
 (37, 'under', 1, 1),
 (38, 'tree.', 3, 1),
 (39, 'kindhearted', 1, 1),
 (40, 'offered', 1, 1),
 (41, 'him', 4, 1),
 (42, 'some', 2, 1),
 (43, 'fruits.', 1, 1),
 (44, 'back', 3, 1),
 (45, 'next', 1, 1),
 (46, 'day', 1, 1),
 (47, 'for'

Finally! Our tables are filled and we can now run SELECT queries against it to pull the data we want. There are two queries you will need to run. 
### SELECT Query One: 
Grab all rows from `word_counts` where the word is "the" and the count is greater than 1.

In [30]:
query_one = "SELECT * FROM word_counts WHERE word = 'the' AND count > 1" #Input SELECT query one here
cur.execute(query_one)
records = cur.fetchall()
for record in records:
    print(record)
    


(4, 'the', 26, 1)
(158, 'the', 18, 2)
(335, 'the', 8, 3)
(425, 'the', 19, 4)


### SELECT Query Two: 
Grab the `story_id`, `story`, and `count` columns where the word is "the". You should use a JOIN statement for this query, and only need to include `story_id` from one table.

In [32]:
# using ALIAS s for stories and wc for word_counts

query_two = "SELECT s.story_id, s.story, wc.count \
    FROM stories s JOIN word_counts wc ON s.story_id = wc.story_id \
    WHERE wc.word = 'the'"   #Input SELECT query two here

cur.execute(query_two)
records = cur.fetchall()
for record in records:
    print(record)

(1, 'The Monkey and the Crocodile:\n\nOnce upon a time, in a forest, there lived a monkey who resided on a jamoon (berry) tree, which was on the banks of a river. In the same forest, there lived a crocodile and his wife. One day, the crocodile came to the banks of the river and rested under the tree. The kindhearted monkey offered him some fruits. The crocodile came back the next day for more fruits, as he loved them. As days passed by, the crocodile and the monkey became good friends.\n\nOne day, the monkey sent some fruits for the crocodile’s wife. She ate the fruits and liked them, but was jealous, as she didn’t like her husband spending time with the monkey. She told her husband, “If the fruits are so juicy, I wonder how sweet the monkey’s heart would be. Get me the heart of the monkey.” The crocodile was not willing to kill his friend, but had no choice.\n\nHe invited the monkey to his house for dinner and that his wife would like to meet him. The monkey was happy, but couldn’t sw

### Part 2 - Soccer Database
Now that we have our first database filled with data, we can move on to the second one. We will be using the `fifa_soccer_dataset.sqlite` file.
Feel free to use either sqlite3 or pandas to run your queries!
If you plan to use pandas, check the `pandas` documentation for how to read in a sqlite database. In particular you can load a sqlite database into a pandas dataframe using the `read_sql_query` function.


In [ ]:
dataset_path = "../../Datasets/fifa_soccer_dataset.sqlite" # Fix your path accordingly

import sqlite3
conn = sqlite3.connect(dataset_path)


If you are using pandas, import it and read in the database. For instance:

In [ ]:
import pandas as pd
# get all tables
df_tables = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", conn)
display(df_tables)
# get all players from Players table
df_players = pd.read_sql_query("SELECT * FROM Player", conn)
display(df_players)


If you prefer to use just sqlite3, you can do that as well. Just make sure to import the library and connect to the database:

In [ ]:
conn = sqlite3.connect(dataset_path)
cur = conn.cursor()
# get all tables
cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cur.fetchall()
for table in tables:
    print(table)
# get all players from Players table
cur.execute("SELECT * FROM Player")
players = cur.fetchall()
for player in players:
    print(player)

**QUESTION 1**

Print the birthday of the player whose name is “Aaron Kuhl”. *Hint: Use ‘Player’ table*

In [ ]:
# YOUR CODE HERE

**QUESTION 2**

Print the number of times the team_fifa_api_id ‘673’ appeared in Team_attribute table. *Hint: Apply GROUP BY clause on team_fifa_api_id attribute*


In [ ]:
# YOUR CODE HERE

**QUESTION 3**

Print country name and league name that have matches on “2014-04-20 00:00:00”. *Hint: Apply join on Match Table and Country table, then Match Table and League Table*


In [ ]:
# YOUR CODE HERE